# Countermeasure to DPA and CPA on AES(SIMULATED)
In the previous notebook, we designed and implemented a countermeasure against SPA. In this notebook you will learn about masking as a countermeasure to protect AES against CPA's. (You will not need to adjust the ssource code yourself, but we have modified the code for you so you just need to compile and flash the new firmware).

**Goals:**
* Learn to use Chipwhisperer projects
* Use masking as countermeasure against CPA on AES
* Test out the countermeasure

#### Prerequisites
- [x] *0_series* notebooks 
- [x] *1_series* notebooks
- [x] *2_series* notebooks
- [x] *3_A - Countermeasure to SPA on RSA(SIMULATED)* notebook

## Chipwhisperer projects
To begin we are going to see how we can use CW projects to perform a CPA very easily. All the code used in notebook `2_B - CPA on AES cryptosystem` is already packaged nicely in the CW api (the api provides an even better verion of the attack). We already ran the projects and the results are automatically saved.

#### Loading the CW project using the api

In [28]:
import chipwhisperer as cw
import numpy as np
proj = cw.open_project("../AES_normal.cwp")
key = "CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')"

#### performing the attack using the api

In [29]:
import chipwhisperer.analyzer as cwa

def CPA_attack(proj):
    leak_model = cwa.leakage_models.sbox_output
    attack = cwa.cpa(proj, leak_model)
    results = attack.run()
    print(results)
    print("The attack reveals the following key guess:\n",end='')
    print(str(bytearray(results.key_guess())),end=' ')

CPA_attack(proj)
print(" ")
print("\nThe real key was:\n"+str(key))

Subkey KGuess Correlation
  00    0x2B    0.92689
  01    0x7E    0.88249
  02    0x15    0.86692
  03    0x16    0.87751
  04    0x28    0.81487
  05    0xAE    0.84567
  06    0xD2    0.80737
  07    0xA6    0.85593
  08    0xAA    0.76701
  09    0xF7    0.86881
  10    0x15    0.93563
  11    0x88    0.84391
  12    0x09    0.70739
  13    0xCF    0.77456
  14    0x4F    0.90720
  15    0x3C    0.91410

The attack reveals the following key guess:
CWbytearray(b'2b 7e 15 16 28 ae d2 a6 aa f7 15 88 09 cf 4f 3c')  

The real key was:
CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')


As you can tell, using the CW project is super easy. The only thing to remember is the kind of attack (what kind of leakage model) we wish to use. In our case it is `leak_model = cwa.leakage_models.sbox_output`. We will use the projects to test our new version of the AES to see if it can still crack it.

## Designing a countermeasure
An important part of the countermeasure is to keep te encryption the same the masked AES firmware already implements the countermeasure and keeps the encryption the same. 

The next step is to go over the changes in the masked AES and finally test if we can crack it. 
The reason we were able to crack the normal AES is that there was a correlation between the powerconsumtion and the data set on the databusses. We could predict what the value of a databit would be if we guessed the correct key. By comparing this to the measured power traces we could quantify the probability of our guess being correct. In order to make it impossible for us to know the value of the databus internally we will use a strategy called masking. Masking means that we perform an operation between the plain text and a randomly generated number. The operation must be choosen so that at the final step we can use that same random number to peel back the mask and reveal the correct encryption. A good operation for this is the `XOR` operation because to reverse an xor with a number you just have to repeat the xor operation with the same number. 
```
byte = 0xF1 = 0b1111 0001
mask = 0x55 = 0b0101 0101

0xF1^0x55 = 0xA4 = 0b1010 0100
0xA4^0x55 = 0xF1 = 0b1111 0001
```
Furthermore when performing linear operations on the masked data the same random number can be used to unmask the data. For example if we `xor` the masked data with a keybyte we can easily get the unmasked xor between the key by unmasking with the same mask:
```
normal: 0xF1 ^ 0x2B        = 0xDA = 0b1101 1010
masked: (0xF1^0x55) ^ 0x2B = 0x8F = 0b0100 1111
unmasking: 0x8F ^ 0x55     = 0xDA
```
This means we can perform the AES operations while not knowing the inner bit values because they depend on the random mask. There is one problem: the SBOX operation does not keep the mask. We can circumvent this problem by unmasking the data before sending it into the SBOX and remasking its output. Just executing those steps sequentially in code is not an option however. The buslines would hold the same data as in the normal AES for a split second. The means the mask would be useless, but we can instead make a different SBOX based on the random mask that already contains this unmasking and remasking of the data. That way the databusses never get the value as in the normal AES because the new SBOX is created ahead of time right after the random mask is made. Let's check out what this would look like:
```
//Sequential (unsafe):
- sbox_input = masked_input ^ mask       //unmask the byte before using the sbox
- sbox_out   = sbox(sbox_input)          //performing sbox operation
- remasked_sbox_output = sbox_out ^ mask //remasking the output

//Masking new SBOX based on mask
for(i=0; i<length(sbox); ++i){
        sbox2[i^mask] = aes_sbox[i]^mask;
	}   

//then just use the following
remasked_sbox_output = sbox2[masked^input]

//EXAMPLE:
sbox[0xDA] = 0x57  //normal aes

sbox2[0xDA^0x55] = sbox2[0x8F] = 0x02 (= sbox[0xDA]^0x55) //masked
0x02 ^ 0x55 = 0x57 //after unmasking is same as normal aes
```
To see how these changes to AES are implemented in the Masked AES you can open the 2 changed source files here: 

[simpleserial-aes.c](http://localhost:8888/edit/IIW/UHasselt%20Chipwhisperer%20notebooks/src/aes_masked/simpleserial-aes/simpleserial-aes.c)

[aes_enc.c](http://localhost:8888/edit/IIW/UHasselt%20Chipwhisperer%20notebooks/src/aes_masked/crypto/avrcryptolib/aes/aes_enc.c)

The original source files of the normal AES you can open here:

[simpleserial-aes.c](http://localhost:8888/edit/chipwhisperer/hardware/victims/firmware/simpleserial-aes/simpleserial-aes.c)

[aes_enc.c](http://localhost:8888/edit/chipwhisperer/hardware/victims/firmware/crypto/avrcryptolib/aes/aes_enc.c)

## Testing the masked AES
The final step is to try and crack our masked AES. We will use a CW project again, but this time it's up to you to write the correct code:

In [32]:
proj = cw.open_project("../AES_masked")

In [33]:
#Start your code here
###START SOLUTION###
CPA_attack(proj)
print(" ")
print("\nThe real key was:\n"+str(key))
###END SOLUTION###

Subkey KGuess Correlation
  00    0x27    0.64263
  01    0x21    0.66350
  02    0x9D    0.63112
  03    0xF2    0.66846
  04    0x6B    0.59610
  05    0xAE    0.60065
  06    0xC9    0.61420
  07    0x51    0.63192
  08    0xE6    0.61918
  09    0xA6    0.61256
  10    0xD1    0.68282
  11    0x37    0.69283
  12    0xAA    0.67155
  13    0x2B    0.63045
  14    0xB7    0.63124
  15    0xC0    0.63472

The attack reveals the following key guess:
CWbytearray(b'27 21 9d f2 6b ae c9 51 e6 a6 d1 37 aa 2b b7 c0')  

The real key was:
CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')


## Conclusion
By using a mask the internal data does not correlate with the predicted data. That way a CPA attack is prevented from finding the secret key.

## THE END
Congratulations on finishing this tutorial series and feel free to experiment further with the Chipwhisperer.